In [ ]:
#!pip install rapidfuzz
import pandas as pd
from rapidfuzz import fuzz

from IPython.display import display, Markdown

#!pip install scikit-learn pandas unidecode nltk

import pandas as pd
import re
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# ====== 1. Crear dataset más amplio y diverso ======
data = [
    ["sinto dor no peito e falta de ar", "alto risco"],
    ["sinto palpitações e tontura", "alto risco"],
    ["houve desmaio súbito durante atividade física", "alto risco"],
    ["apresento febre alta e dificuldade para respirar", "alto risco"],
    ["perdi a consciência por alguns segundos", "alto risco"],
    ["me sinto extremamente cansado e fraco", "alto risco"],
    ["tenho dor intensa na cabeça e náusea", "alto risco"],
    ["leve dor nas costas após exercício", "baixo risco"],
    ["meu braço dói levemente", "baixo risco"],
    ["sinto desconforto na garganta", "baixo risco"],
    ["tenho cansaço leve, mas sem outros sintomas", "baixo risco"],
    ["leve dor de cabeça ocasional", "baixo risco"],
    ["apenas um incômodo nas articulações", "baixo risco"],
    ["sinto formigamento leve nas mãos", "baixo risco"],
    ["leve desconforto estomacal após refeições", "baixo risco"]
]

df = pd.DataFrame(data, columns=["frase", "situacao"])
df.to_csv("frases_risco.csv", index=False, encoding="utf-8")

# ====== 2. Preprocesamiento de texto ======
def preprocess(text):
    text = text.lower()                      # Minúsculas
    text = unidecode.unidecode(text)         # Quitar acentos
    text = re.sub(r'[^a-z\s]', '', text)    # Quitar puntuación
    stop_words = set(stopwords.words('portuguese'))
    words = [w for w in text.split() if w not in stop_words]
    return " ".join(words)

df['frase_clean'] = df['frase'].apply(preprocess)

# ====== 3. Separar features y labels ======
X = df['frase_clean']
y = df['situacao']

# ====== 4. Transformar frases en vectores TF-IDF ======
vectorizer = TfidfVectorizer()
X_vect = vectorizer.fit_transform(X)

# ====== 5. Dividir en entrenamiento y prueba ======
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.3, random_state=42)

# ====== 6. Entrenar modelo Random Forest ======
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# ====== 7. Evaluar desempeño ======
y_pred = modelo.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

# =========================================================================
# ====== 8. Testar con nuevas frases y MOSTRAR RESULTADO EM TABELA ======
# =========================================================================

novas_frases = [
    "sinto dor intensa no peito e dificuldade para respirar",
    "leve dor de cabeça ocasional",
    "desmaio repentino durante corrida",
    "formigamento nas mãos e cansaço",
    "dor leve no estômago após almoço"
]

novas_frases_clean = [preprocess(f) for f in novas_frases]
novas_vect = vectorizer.transform(novas_frases_clean)
predicoes = modelo.predict(novas_vect)
probabilidades = modelo.predict_proba(novas_vect)

resultados_list = [] # Lista para armazenar os dados de cada linha da tabela

# Iterar sobre os resultados e construir a lista
for frase, pred, prob in zip(novas_frases, predicoes, probabilidades):
    # Dicionário com as classes do modelo e suas probabilidades
    prob_dict = {classe: round(p*100,2) for classe, p in zip(modelo.classes_, prob)}

    # Adicionar linha de dados à lista
    resultados_list.append({
        "Frase de Teste": frase,
        "Risco Previsto": pred.upper(),
        "Prob. Alto Risco (%)": prob_dict.get('alto risco', 0),
        "Prob. Baixo Risco (%)": prob_dict.get('baixo risco', 0)
    })

# Criar o DataFrame (Tabela)
df_resultados = pd.DataFrame(resultados_list)

# Exibir o título e a tabela formatada no Colab
display(Markdown("---"))
display(Markdown('## Resultados da Classificação de Novas Frases'))
display(df_resultados)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Acurácia: 0.8

Relatório de classificação:
              precision    recall  f1-score   support

  alto risco       0.67      1.00      0.80         2
 baixo risco       1.00      0.67      0.80         3

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5



---

## Resultados da Classificação de Novas Frases

,Frase de Teste,Risco Previsto,Prob. Alto Risco (%),Prob. Baixo Risco (%)
0,sinto dor intensa no peito e dificuldade para ...,ALTO RISCO,73.0,27.0
1,leve dor de cabeça ocasional,BAIXO RISCO,44.0,56.0
2,desmaio repentino durante corrida,ALTO RISCO,62.0,38.0
3,formigamento nas mãos e cansaço,BAIXO RISCO,47.0,53.0
4,dor leve no estômago após almoço,BAIXO RISCO,27.0,73.0
